In [2]:
import asyncio
import json

from aptos_sdk.account import Account
from aptos_sdk.account_address import AccountAddress
from aptos_sdk.aptos_token_client import (
    AptosTokenClient,
    Collection,
    Object,
    PropertyMap,
    ReadObject,
    Token,
    Property
)
from aptos_sdk.async_client import FaucetClient, RestClient

import os
from dotenv import load_dotenv
load_dotenv()

# :!:>section_1
NODE_URL = os.getenv("APTOS_NODE_URL", "https://fullnode.devnet.aptoslabs.com/v1")
FAUCET_URL = os.getenv(
    "APTOS_FAUCET_URL",
    "https://faucet.devnet.aptoslabs.com",
)  # <:!:section_1

In [3]:
def get_owner(obj: ReadObject) -> AccountAddress:
    return obj.resources[Object].owner

# :!:>get_token_data
async def get_token_data(
    token_client: AptosTokenClient, token_addr: AccountAddress
) -> dict[str, str]:
    token = (await token_client.read_object(token_addr)).resources[Token]
    return {
        "collection": str(token.collection),
        "description": str(token.description),
        "name": str(token.name),
        "uri": str(token.uri),
        "index": str(token.index),
    }  # <:!:get_token_data

# :!:>section_6
async def get_collection_data(
    token_client: AptosTokenClient, collection_addr: AccountAddress
) -> dict[str, str]:
    collection = (await token_client.read_object(collection_addr)).resources[Collection]
    return {
        "creator": str(collection.creator),
        "name": str(collection.name),
        "description": str(collection.description),
        "uri": str(collection.uri),
    }  # <:!:section_6



In [5]:
# Create API and faucet clients.
# :!:>section_1a
rest_client = RestClient(NODE_URL)
faucet_client = FaucetClient(FAUCET_URL, rest_client)  # <:!:section_1a

# Create client for working with the token module.
# :!:>section_1b
token_client = AptosTokenClient(rest_client)  # <:!:section_1b

# :!:>section_2
PrivateKey = os.getenv("PrivateKey")

alice = Account.load_key(PrivateKey)

collection_name = "idol-demo-dev"
token_name = "ikun"

# :!:>owners
owners = {str(alice.address()): "Alice"}  # <:!:owners

print("\n=== Addresses ===")
print(f"Alice: {alice.address()}")


=== Addresses ===
Alice: 0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81


In [42]:
# :!:>section_3
# alice_fund = faucet_client.fund_account(alice.address(), 100_000_000)  # <:!:section_3
# await asyncio.gather(*[alice_fund])

print("\n=== Initial Coin Balances ===")
alice_balance = rest_client.account_balance(alice.address())
[alice_balance] = await asyncio.gather(*[alice_balance])
print(f"Alice: {alice_balance}")

print("\n=== Creating Collection and Token ===")

# :!:>section_4
txn_hash = await token_client.create_collection(
    alice,
    "idol's simple collection",
    1,
    collection_name,
    "https://aptos.dev",
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    True,
    0,
    1,
)  # <:!:section_4
await rest_client.wait_for_transaction(txn_hash)

collection_addr = AccountAddress.for_named_collection(
    alice.address(), collection_name
)


=== Initial Coin Balances ===
Alice: 199895400

=== Creating Collection and Token ===


In [43]:
# :!:>section_5
txn_hash = await token_client.mint_token(
    alice,
    collection_name,
    "idol's simple token",
    token_name,
    "https://raw.githubusercontent.com/realTaki/Aptos-hack-round-1/main/NFT/ikun.png",
    PropertyMap([]),
)  # <:!:section_5
await rest_client.wait_for_transaction(txn_hash)

minted_tokens = await token_client.tokens_minted_from_transaction(txn_hash)
assert len(minted_tokens) == 1

token_addr = minted_tokens[0]


In [16]:
collection_addr = AccountAddress.for_named_collection(
    alice.address(), collection_name
)
collection_data = await get_collection_data(token_client, collection_addr)
print(
    "\nCollection data: "
    + json.dumps({"address": str(collection_addr), **collection_data}, indent=4)
)



Collection data: {
    "address": "0xc2ae9b9441c39ac729404fe898d9c86a3625c0ea29549476cecb5941e0ca3313",
    "creator": "0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81",
    "name": "idol-demo",
    "description": "Alice's simple collection",
    "uri": "https://aptos.dev"
}


In [ ]:
# Check the owner
# :!:>section_7
obj_resources = await token_client.read_object(token_addr)
owner = str(get_owner(obj_resources))
print(f"\nToken owner: {owners[owner]}")  # <:!:section_7
token_data = await get_token_data(token_client, token_addr)
print(
    "Token data: "
    + json.dumps(
        {"address": str(token_addr), "owner": owner, **token_data}, indent=4
    )
)

# Read the object owner one last time
# :!:>section_11
obj_resources = await token_client.read_object(token_addr)
print(f"Token owner: {owners[str(get_owner(obj_resources))]}\n")  # <:!:section_11

await rest_client.close()

In [6]:
token_data = await get_token_data(token_client, "0x8222f8151b87dc006dde71feaa0f734b161e0c84ad8dde43a57bc0dd2243e0a5")
print(
    "Token data: "
    + json.dumps(
       token_data
    )
)

Token data: {"collection": "0xaa93b9da1e3ca3bae689040ee8c0b7c1dee4c82c5ee4e350c74195bc267b85cb", "description": "idol's simple token", "name": "ikun", "uri": "https://raw.githubusercontent.com/realTaki/Aptos-hack-round-1/main/NFT/ikun.png", "index": "1"}


In [44]:
# token_addr = AccountAddress(b"0x57c3961f9c19be3ace9c8d47b28046860007467878fe94733d8ebb539f412901")
# token_addr
obj_resources = await token_client.read_object("0x8222f8151b87dc006dde71feaa0f734b161e0c84ad8dde43a57bc0dd2243e0a5")
print(obj_resources)

ReadObject
	0x1::object::ObjectCore: Object[allow_ungated_transfer: True, owner: 0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81]
	0x4::property_map::PropertyMap: PropertyMap[]
	0x4::token::Token: Token[collection: 0xaa93b9da1e3ca3bae689040ee8c0b7c1dee4c82c5ee4e350c74195bc267b85cb, index: 1, description: idol's simple token, name: ikun, uri: https://raw.githubusercontent.com/realTaki/Aptos-hack-round-1/main/NFT/ikun.png]


In [46]:
collection_data = await get_collection_data(token_client, "0xaa93b9da1e3ca3bae689040ee8c0b7c1dee4c82c5ee4e350c74195bc267b85cb")
print(
    "\nCollection data: "
    + json.dumps(collection_data, indent=4)
)


Collection data: {
    "creator": "0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81",
    "name": "idol-demo-dev",
    "description": "idol's simple collection",
    "uri": "https://aptos.dev"
}


In [35]:
token_addr = AccountAddress.from_str("0x57c3961f9c19be3ace9c8d47b28046860007467878fe94733d8ebb539f412901")
token_addr

0x57c3961f9c19be3ace9c8d47b28046860007467878fe94733d8ebb539f412901

In [37]:
zero_addr = AccountAddress.from_str("0x000")
print("\n=== Transferring the token to Bob ===")
txn_hash = await token_client.transfer_token(
    alice,
    token_addr,
    zero_addr,
)
await rest_client.wait_for_transaction(txn_hash)  


=== Transferring the token to Bob ===


In [38]:
# :!:>section_5
txn_hash = await token_client.mint_token(
    alice,
    collection_name,
    "idol-demo's simple token",
    token_name,
    "https://raw.githubusercontent.com/realTaki/Aptos-hack-round-1/main/NFT/ikun.png",
    PropertyMap([]),
)  # <:!:section_5
await rest_client.wait_for_transaction(txn_hash)

AssertionError: {"version":"535406848","hash":"0x79e012f31d87acd1f2188f7353e58ecae30d64228240e0af69352db9d298236d","state_change_hash":"0x5f4aabce7206ecbfa0a9e3b2508d4eeadf74c34b4613e2c2000dceebbd1ff6e6","event_root_hash":"0x93429c9f85042bf4e7b5be0e3b4ddad17c2202a9e58d17883328c7404fcce2a6","state_checkpoint_hash":null,"gas_used":"7","success":false,"vm_status":"Move abort in 0x4::collection: ECOLLECTION_SUPPLY_EXCEEDED(0x20002): The collection has reached its supply and no more tokens can be minted, unless some are burned","accumulator_root_hash":"0x3a7ac7c09426005df695d95deea17c4963032dd234dd210afcbc234748225d54","changes":[{"address":"0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81","state_key_hash":"0xae64aedc2e651f20b9c510186523456f8cf4da37ea480b25dfa7bc0a4821bbaf","data":{"type":"0x1::coin::CoinStore<0x1::aptos_coin::AptosCoin>","data":{"coin":{"value":"199895900"},"deposit_events":{"counter":"2","guid":{"id":{"addr":"0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9

In [36]:
txn_hash = await token_client.update_token_property(alice, token_addr, Property(name="uri", value="https://raw.githubusercontent.com/realTaki/Aptos-hack-round-1/main/NFT/ikun.png", property_type="0x1::string::String"))
await rest_client.wait_for_transaction(txn_hash)

AssertionError: {"version":"535395212","hash":"0x67ff922436d5eee13e61de9c5e0b0b06ca46cb0b81c4bca8683ff16e7d7374af","state_change_hash":"0x033b3a733d2c61a04f25de9655d0a2925eea257b9a621f70b589677b3de5c339","event_root_hash":"0x93429c9f85042bf4e7b5be0e3b4ddad17c2202a9e58d17883328c7404fcce2a6","state_checkpoint_hash":null,"gas_used":"7","success":false,"vm_status":"Move abort in 0x1::simple_map: EKEY_NOT_FOUND(0x10002): Map key is not found","accumulator_root_hash":"0x5beb943ddbce79d5a883969d13b5c21b89540f4a818b76a715998bbac695faae","changes":[{"address":"0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81","state_key_hash":"0xae64aedc2e651f20b9c510186523456f8cf4da37ea480b25dfa7bc0a4821bbaf","data":{"type":"0x1::coin::CoinStore<0x1::aptos_coin::AptosCoin>","data":{"coin":{"value":"199897100"},"deposit_events":{"counter":"2","guid":{"id":{"addr":"0x81edff0d5fabe3cbbf9ccb92c18cc1664eb38672e9262c9c8bd08c0bed258d81","creation_num":"2"}}},"frozen":false,"withdraw_events":{"counte